In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

data = pd.read_csv('home_data_raw.csv')
years = pd.read_csv('years_built.csv')

### Drop duplicates and merge

In [2]:
data.drop_duplicates(subset='detailUrl', inplace=True)
years.drop_duplicates(inplace=True)

df = pd.merge(data, years)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    421 non-null    int64  
 1   zipcode       421 non-null    int64  
 2   city          421 non-null    object 
 3   latitude      392 non-null    float64
 4   longitude     392 non-null    float64
 5   price         421 non-null    float64
 6   bathrooms     369 non-null    float64
 7   bedrooms      367 non-null    float64
 8   livingArea    363 non-null    float64
 9   homeType      421 non-null    object 
 10  lotAreaValue  290 non-null    float64
 11  lotAreaUnit   290 non-null    object 
 12  detailUrl     421 non-null    object 
 13  years         421 non-null    object 
dtypes: float64(7), int64(2), object(5)
memory usage: 46.2+ KB
None


### Drop and rename colums

In [3]:
df.drop(columns=['Unnamed: 0', 'latitude', 'longitude'], inplace=True)
df.rename(columns={'zipcode': 'zip', 'bathrooms': 'baths', 'bedrooms': 'beds',
                   'livingArea': 'home_sqft', 'homeType': 'type', 'detailUrl': 'url',
                   'years': 'year_built'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           421 non-null    int64  
 1   city          421 non-null    object 
 2   price         421 non-null    float64
 3   baths         369 non-null    float64
 4   beds          367 non-null    float64
 5   home_sqft     363 non-null    float64
 6   type          421 non-null    object 
 7   lotAreaValue  290 non-null    float64
 8   lotAreaUnit   290 non-null    object 
 9   url           421 non-null    object 
 10  year_built    421 non-null    object 
dtypes: float64(5), int64(1), object(5)
memory usage: 36.3+ KB


### Change non-date years to np.nan

In [4]:
df.loc[df[df['year_built']=='data'].index, 'year_built'] = np.nan
df['year_built'] = df['year_built'].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           421 non-null    int64  
 1   city          421 non-null    object 
 2   price         421 non-null    float64
 3   baths         369 non-null    float64
 4   beds          367 non-null    float64
 5   home_sqft     363 non-null    float64
 6   type          421 non-null    object 
 7   lotAreaValue  290 non-null    float64
 8   lotAreaUnit   290 non-null    object 
 9   url           421 non-null    object 
 10  year_built    368 non-null    float64
dtypes: float64(6), int64(1), object(4)
memory usage: 36.3+ KB


### Drop all 'LOT' home types

In [5]:
df[df['year_built'].isnull()]['type'].value_counts()

type
LOT    53
Name: count, dtype: int64

In [6]:
df.drop(df[df['type']=='LOT'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           368 non-null    int64  
 1   city          368 non-null    object 
 2   price         368 non-null    float64
 3   baths         367 non-null    float64
 4   beds          367 non-null    float64
 5   home_sqft     363 non-null    float64
 6   type          368 non-null    object 
 7   lotAreaValue  241 non-null    float64
 8   lotAreaUnit   241 non-null    object 
 9   url           368 non-null    object 
 10  year_built    368 non-null    float64
dtypes: float64(6), int64(1), object(4)
memory usage: 31.8+ KB


### Drop/Replace null and unwanted 0 values

In [7]:
df.describe()

,zip,price,baths,beds,home_sqft,lotAreaValue,year_built
count,368.000000,3.680000e+02,367.000000,367.000000,363.000000,241.000000,368.000000
mean,91745.654891,1.058524e+06,2.880109,3.370572,2128.972452,3656.895669,1985.763587
std,30.920006,9.714517e+05,1.347660,1.378605,1404.620864,3920.214915,28.575748
min,91702.000000,7.500000e+04,0.000000,0.000000,0.000000,0.251492,1907.000000
25%,91722.000000,6.184000e+05,2.000000,3.000000,1251.000000,0.516460,1962.000000
50%,91741.000000,7.990000e+05,3.000000,3.000000,1637.000000,1784.000000,1979.000000
75%,91773.000000,1.178750e+06,3.000000,4.000000,2341.500000,7388.000000,2020.250000
max,91792.000000,9.495000e+06,8.000000,9.000000,8863.000000,10808.000000,2025.000000


In [8]:
df[(df['baths'].isnull()) | (df['baths']==0) | (df['beds'].isnull()) | (df['beds']==0)]

,zip,city,price,baths,beds,home_sqft,type,lotAreaValue,lotAreaUnit,url,year_built
118,91724,Covina,79000.0,1.0,0.0,255.0,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/19548-E-Cypress-St-4-Covina-CA-91724/2068217436_zpid/,2021.0
217,91702,Azusa,1099000.0,0.0,NaN,NaN,MULTI_FAMILY,7254.00,sqft,https://www.zillow.com/homedetails/329-N-Soldano-Ave-Azusa-CA-91702/21626494_zpid/,1981.0
219,91702,Azusa,1175000.0,NaN,6.0,3554.0,MULTI_FAMILY,7419.00,sqft,https://www.zillow.com/homedetails/603-N-Rockvale-Ave-Azusa-CA-91702/21626862_zpid/,1963.0
249,91773,San Dimas,680000.0,1.0,0.0,575.0,SINGLE_FAMILY,0.35,acres,https://www.zillow.com/homedetails/127-Ashvale-Dr-San-Dimas-CA-91773/21649219_zpid/,1951.0
297,91750,La Verne,399900.0,0.0,0.0,0.0,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/4095-Fruit-St-702-La-Verne-CA-91750/2056358352_zpid/,2023.0


In [9]:
# Drop samples with no reported 'baths', 'beds', or 'home_sqft'
df.drop([217, 297], inplace=True)

In [10]:
# Find strongest correlations to 'baths' to estimate NaN values
df.corr(numeric_only=True)['baths'].sort_values(ascending=False)

baths           1.000000
home_sqft       0.854795
price           0.745414
beds            0.674933
year_built      0.327768
zip             0.121733
lotAreaValue   -0.373925
Name: baths, dtype: float64

In [11]:
# Use linear regression to estimate 'baths' based on 'home_sqft'
from sklearn.linear_model import LinearRegression

df_baths = df[['home_sqft', 'baths']].dropna()

lr = LinearRegression()
lr.fit(df_baths['home_sqft'].values.reshape(-1, 1), df_baths['baths'])
y_predict = lr.predict(np.array(3554).reshape(1, -1))
df.loc[219, 'baths'] = np.round(y_predict)
df.loc[219]

zip                                                                                           91702
city                                                                                          Azusa
price                                                                                     1175000.0
baths                                                                                           4.0
beds                                                                                            6.0
home_sqft                                                                                    3554.0
type                                                                                   MULTI_FAMILY
lotAreaValue                                                                                 7419.0
lotAreaUnit                                                                                    sqft
url             https://www.zillow.com/homedetails/603-N-Rockvale-Ave-Azusa-CA-91702/21626862_zpid/


#### Drop/replace null and unwanted 0 'home_sqft' values

In [12]:
nulls_sqft = df[df['home_sqft'].isnull() | (df['home_sqft']==0)]
nulls_sqft

,zip,city,price,baths,beds,home_sqft,type,lotAreaValue,lotAreaUnit,url,year_built
75,91722,Covina,137000.0,2.0,2.0,NaN,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1045-N-Azusa-Ave-Covina-CA-91722/136522189_zpid/,1968.0
79,91722,Covina,139999.0,2.0,3.0,NaN,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1380-N-Citrus-Ave-TRAILER-D9-Covina-CA-91722/136705735_zpid/,1966.0
117,91722,Covina,131000.0,2.0,3.0,0.0,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1045-N-Azusa-Ave-58-Covina-CA-91722/136705814_zpid/,1999.0
128,91724,Covina,90000.0,1.0,1.0,NaN,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/716-N-Grand-Ave-TRLR-J1-Covina-CA-91724/2103264204_zpid/,1967.0
351,91706,Baldwin Park,79979.0,1.0,1.0,NaN,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1661-Puente-Ave-TRAILER-26-Baldwin-Park-CA-91706/136706318_zpid/,1976.0


In [13]:
meds_sqft = df.groupby(['baths', 'beds'])['home_sqft'].median()
meds_sqft

baths  beds
1.0    0.0      415.0
       1.0      533.0
       2.0      832.0
       3.0     1218.0
2.0    1.0      935.0
       2.0     1244.0
       3.0     1298.5
       4.0     1701.0
       5.0     2103.5
3.0    1.0     1683.0
       2.0     1318.0
       3.0     1639.5
       4.0     2124.0
       5.0     2876.5
       6.0     1960.0
       7.0     1749.0
4.0    1.0     1573.0
       3.0     1629.0
       4.0     2900.0
       5.0     4119.5
       6.0     3585.0
       7.0     3032.5
       8.0     3600.0
5.0    4.0     4686.0
       5.0     4741.0
       6.0     1647.0
       8.0     3690.0
6.0    4.0     2870.0
       5.0     5937.0
       6.0     5511.0
       7.0     7349.0
       8.0     5604.0
7.0    4.0     6640.0
       5.0     6022.5
       6.0     5556.0
       8.0     5745.0
       9.0     4050.0
8.0    8.0     5182.0
Name: home_sqft, dtype: float64

In [14]:
for i in nulls_sqft.index:
    sqft = meds_sqft[nulls_sqft.loc[i, 'baths'], nulls_sqft.loc[i, 'beds']]
    print(sqft)
    nulls_sqft.loc[i, 'home_sqft'] = sqft

1244.0
1298.5
1298.5
533.0
533.0


In [15]:
nulls_sqft

,zip,city,price,baths,beds,home_sqft,type,lotAreaValue,lotAreaUnit,url,year_built
75,91722,Covina,137000.0,2.0,2.0,1244.0,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1045-N-Azusa-Ave-Covina-CA-91722/136522189_zpid/,1968.0
79,91722,Covina,139999.0,2.0,3.0,1298.5,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1380-N-Citrus-Ave-TRAILER-D9-Covina-CA-91722/136705735_zpid/,1966.0
117,91722,Covina,131000.0,2.0,3.0,1298.5,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1045-N-Azusa-Ave-58-Covina-CA-91722/136705814_zpid/,1999.0
128,91724,Covina,90000.0,1.0,1.0,533.0,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/716-N-Grand-Ave-TRLR-J1-Covina-CA-91724/2103264204_zpid/,1967.0
351,91706,Baldwin Park,79979.0,1.0,1.0,533.0,MANUFACTURED,NaN,NaN,https://www.zillow.com/homedetails/1661-Puente-Ave-TRAILER-26-Baldwin-Park-CA-91706/136706318_zpid/,1976.0


In [16]:
df[df['home_sqft'].isnull() | (df['home_sqft']==0)] = nulls_sqft
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 366 entries, 0 to 367
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           366 non-null    int64  
 1   city          366 non-null    object 
 2   price         366 non-null    float64
 3   baths         366 non-null    float64
 4   beds          366 non-null    float64
 5   home_sqft     366 non-null    float64
 6   type          366 non-null    object 
 7   lotAreaValue  240 non-null    float64
 8   lotAreaUnit   240 non-null    object 
 9   url           366 non-null    object 
 10  year_built    366 non-null    float64
dtypes: float64(6), int64(1), object(4)
memory usage: 42.4+ KB


### Drop/Replace null 'lotAreaValue'

In [17]:
print(df[(df['lotAreaValue'].isnull()) & (df['type']=='SINGLE_FAMILY')])

       zip         city      price  baths  beds  home_sqft           type  \
24   91790  West Covina   654000.0    3.0   3.0     1491.0  SINGLE_FAMILY   
27   91790  West Covina   779000.0    3.0   3.0     1465.0  SINGLE_FAMILY   
50   91790  West Covina   880000.0    3.0   4.0     2125.0  SINGLE_FAMILY   
53   91790  West Covina   613000.0    3.0   2.0     1216.0  SINGLE_FAMILY   
55   91790  West Covina   858000.0    3.0   4.0     1955.0  SINGLE_FAMILY   
56   91790  West Covina   714000.0    3.0   3.0     1791.0  SINGLE_FAMILY   
59   91791  West Covina   844990.0    3.0   3.0     1483.0  SINGLE_FAMILY   
61   91791  West Covina   909990.0    3.0   3.0     1637.0  SINGLE_FAMILY   
62   91791  West Covina   994990.0    3.0   4.0     1826.0  SINGLE_FAMILY   
63   91791  West Covina   844990.0    3.0   3.0     1483.0  SINGLE_FAMILY   
64   91791  West Covina   994990.0    3.0   4.0     1826.0  SINGLE_FAMILY   
65   91791  West Covina   909990.0    3.0   3.0     1637.0  SINGLE_FAMILY   

In [18]:
# Drop samples with 'buildable plans'
df.drop(df[(df['lotAreaValue'].isnull()) & (df['type']=='SINGLE_FAMILY')].index, inplace=True)

In [19]:
# Set null 'lotAreaValue's equal to 'home_sqft
null_lots = df[df['lotAreaValue'].isnull()]
df.loc[null_lots.index, 'lotAreaValue'] = df['home_sqft']
df.loc[df[df['lotAreaUnit'].isnull()].index, 'lotAreaUnit'] = 'sqft'
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 343 entries, 0 to 367
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           343 non-null    int64  
 1   city          343 non-null    object 
 2   price         343 non-null    float64
 3   baths         343 non-null    float64
 4   beds          343 non-null    float64
 5   home_sqft     343 non-null    float64
 6   type          343 non-null    object 
 7   lotAreaValue  343 non-null    float64
 8   lotAreaUnit   343 non-null    object 
 9   url           343 non-null    object 
 10  year_built    343 non-null    float64
dtypes: float64(6), int64(1), object(4)
memory usage: 40.3+ KB


### Find 'lot_sqft' for all homes

In [20]:
# Make a new column 'lot_sqft'
df['lot_sqft'] = df['lotAreaValue']

In [21]:
# Convert homes with acres to sqft
df.loc[df[df['lotAreaUnit']=='acres'].index, 'lot_sqft'] = df['lotAreaValue'] * 43560
df.describe()

,zip,price,baths,beds,home_sqft,lotAreaValue,year_built,lot_sqft
count,343.000000,3.430000e+02,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000
mean,91743.798834,1.006613e+06,2.845481,3.376093,2075.297376,2947.008327,1983.172012,17542.232292
std,30.366722,9.108668e+05,1.323219,1.387585,1360.814541,3452.838027,27.787774,39834.136131
min,91702.000000,7.500000e+04,1.000000,0.000000,255.000000,0.251492,1907.000000,255.000000
25%,91722.000000,6.059475e+05,2.000000,3.000000,1242.500000,0.873783,1961.000000,1626.000000
50%,91741.000000,7.800000e+05,3.000000,3.000000,1618.000000,1440.000000,1977.000000,7467.000000
75%,91773.000000,1.162500e+06,3.000000,4.000000,2287.000000,6288.000000,2014.500000,15583.500000
max,91792.000000,9.495000e+06,8.000000,9.000000,8863.000000,10808.000000,2025.000000,340033.716000


In [22]:
# Set condo and townhouse 'lot_sqft' equal to 'home_sqft'
df.loc[df[(df['type']=='CONDO') | (df['type']=='TOWNHOUSE')].index, 'lot_sqft'] = df['home_sqft']

In [23]:
df.drop(columns=['lotAreaValue', 'lotAreaUnit'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   zip         343 non-null    int64  
 1   city        343 non-null    object 
 2   price       343 non-null    float64
 3   baths       343 non-null    float64
 4   beds        343 non-null    float64
 5   home_sqft   343 non-null    float64
 6   type        343 non-null    object 
 7   url         343 non-null    object 
 8   year_built  343 non-null    float64
 9   lot_sqft    343 non-null    float64
dtypes: float64(6), int64(1), object(3)
memory usage: 26.9+ KB


In [24]:
df.describe()

,zip,price,baths,beds,home_sqft,year_built,lot_sqft
count,343.000000,3.430000e+02,343.000000,343.000000,343.000000,343.000000,343.000000
mean,91743.798834,1.006613e+06,2.845481,3.376093,2075.297376,1983.172012,10866.520921
std,30.366722,9.108668e+05,1.323219,1.387585,1360.814541,27.787774,22403.134278
min,91702.000000,7.500000e+04,1.000000,0.000000,255.000000,1907.000000,255.000000
25%,91722.000000,6.059475e+05,2.000000,3.000000,1242.500000,1961.000000,1496.500000
50%,91741.000000,7.800000e+05,3.000000,3.000000,1618.000000,1977.000000,6534.000000
75%,91773.000000,1.162500e+06,3.000000,4.000000,2287.000000,2014.500000,11161.500000
max,91792.000000,9.495000e+06,8.000000,9.000000,8863.000000,2025.000000,225013.536000


In [25]:
df.to_csv('home_data_clean.csv')